- Exercici 1<br>
Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium

### Exercice design
In this exercici i'll try to design a webscrapper that can acces the bolsamadrid website, and return the hisoric information of the value of their assets for the different companies that use it as a their trade-center.<br>
For this the program will ask which company you want the historic from and the period of time you want. <br>

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.options import Options

In [4]:
from time import sleep, ctime

### Creem tot el nescesari per obtenir el driver

In this exercice i'll try to work with python classes to familiarize myself with them. 

For the input the name has to be put with the same format as in the page, for example atresmedia won't be accepted if the input is atres media, since i change the input to uppercase it doesn't matter if it's written in lower case. 

In [82]:
path = 'C:\\Program Files (x86)\\chromedriver.exe'
bolsa_page = 'https://www.bolsamadrid.es/esp/aspx/Portada/Portada.aspx'

In [83]:
class histCompany():
    def __init__(self):
        #Initialize the driver, asks for company name and date to start the search from
        
        opts = Options()
        opts.headles = True
        self.driver = webdriver.Chrome(path, options = opts)
        self.driver.get(bolsa_page)
        self.comp_name = input('Name of the Company: ').upper()
        self.start_date = input('First day of info (DD-MM-YEAR)')
        
        
    def companyList(self):
        #Gets the page to the list of companies it could all be put within 1 method but i wanted to try to make more than 1
        empreses = self.driver.find_element_by_link_text('EMPRESAS COTIZADAS')
        empreses.click()
        sleep(1)
        llista = self.driver.find_element_by_link_text('Lista de Empresas')
        llista.click()
        
    def companyHist(self):
        #Gets the page source so that we can use its information to create conditions
        i = 0
        
        while i <= 6:
            check = self.driver.page_source
        #Creates a while loop to run through the pages of the companies names (6 pages)
        
            if self.comp_name not in check:
            #Checks if the name is on the current page source, if not it increases the i and goes into the next page
                page = self.driver.find_element_by_id('ctl00_Contenido_SiguientesAbj')
                page.click()
                i +=1
            
            else:
            #Does pretty much everything, gets you to the page of the company you want, to the historic information of that company
            #And does the scrapping of the historic and creates a df with all the information requested. 
            
                empresa = self.driver.find_element_by_partial_link_text(self.comp_name)
                empresa.click()
                sleep(1)
                inf_hist = self.driver.find_element_by_link_text('Inf. Histórica')
                inf_hist.click()
                
                #Clears the date boxes and fills them with the input info from earlier.
                dia = self.driver.find_element_by_id('ctl00_Contenido_Desde_Dia')
                dia.clear()
                dia.send_keys(self.start_date[0:2])
                mes = self.driver.find_element_by_id('ctl00_Contenido_Desde_Mes')
                mes.clear()
                mes.send_keys(self.start_date[3:5])
                anys = self.driver.find_element_by_id('ctl00_Contenido_Desde_Año')
                anys.clear()
                anys.send_keys(self.start_date[6:10])
                buscar = self.driver.find_element_by_id('ctl00_Contenido_Buscar')
                buscar.click()
                
                #creates and empty df on which we will append the values we get 
                hist = pd.DataFrame()
                
                while 'Siguientes' in self.driver.page_source:
                #While loop that goes through the different pages of the historic information appending the information of each page to the DF
                    dades = self.driver.find_element_by_id('ctl00_Contenido_tblDatos')
                    dades = (dades.text).split('\n')
                    dades_2 = []
                    dades_3 = []
                    for i in range(len(dades)):
                        dades_2.append(dades[i].replace(',',''))
                    for i in range(len(dades_2)):
                        dades_3.append(dades_2[i].split(' '))
                    df_dades = pd.DataFrame(dades_3, columns = dades_3[0])
                    df_dades.drop(0, inplace = True)
                    hist = hist.append(df_dades, ignore_index = True)
                    siguiente = self.driver.find_element_by_id('ctl00_Contenido_SiguientesAbj')
                    siguiente.click()
                # We finally add the information from the last page since it doesn't have a next button which we used as condition 
                # we do it apart
                dades = self.driver.find_element_by_id('ctl00_Contenido_tblDatos')
                dades = (dades.text).split('\n')
                dades_2 = []
                dades_3 = []
                for i in range(len(dades)):
                    dades_2.append(dades[i].replace(',',''))
                for i in range(len(dades_2)):
                    dades_3.append(dades_2[i].split(' '))
                df_dades = pd.DataFrame(dades_3, columns = dades_3[0])
                df_dades.drop(0, inplace = True)
                hist = hist.append(df_dades, ignore_index = True)
                return hist
                break;
                   


    #def companyPage(self, comp_name)

In [84]:
atres_info = histCompany()

Name of the Company: atresmedia
First day of info (DD-MM-YEAR)02-05-2020


In [85]:
atres_info.companyList()

In [86]:
df = atres_info.companyHist()

In [87]:
df

,Fecha,Cierre,Referencia,Volumen,Efectivo,Último,Máximo,Mínimo,Medio
0,23/09/2020,22000,22360,414.688,931.00995,22000,23400,22000,22451
1,24/09/2020,22100,22000,267.622,591.47807,22100,22560,21600,22101
2,25/09/2020,21700,22100,578.277,1.271.40805,21700,22700,21620,21855
3,28/09/2020,22660,21700,258.730,577.29230,22660,22680,21800,22313
4,29/09/2020,21880,22660,209.738,467.29464,21880,22760,21880,22280
...,...,...,...,...,...,...,...,...,...
252,16/09/2021,35520,35420,159.494,566.94215,35520,35860,35000,35546
253,17/09/2021,35520,35520,212.344,753.02903,35520,35800,35220,35463
254,20/09/2021,34740,35520,296.392,1.032.98978,34740,35300,34340,34852
255,21/09/2021,34700,34740,167.764,586.45849,34700,35500,34620,34957


### Conclusions
We have a working program, that can extract the historic information of any given company on the plataform. In this case it's a simple program that just extracts historic information, but the same could be done with the different parameters of information that the page gives us. But it would be a redundancy to do it now given the matter of the exercice. We could add different inputs asking for the specific information you want and also for the end date in case you didn't want it up until the current date. But it wouldn't be much interesting to do that for now. 

Following this exercice I convert the df to a excel file to do the next task. 

In [89]:
df.to_excel('df_bolsa.xlsx')